In [3]:
# Install required dependencies
import sys
import subprocess

def install_package(package, version=None):
    """Install a package with specific version if needed."""
    if version:
        package_with_version = f"{package}=={version}"
    else:
        package_with_version = package
    
    print(f"Installing {package_with_version}...")
    # Allow all packages to install their dependencies
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_with_version])
    print(f"Successfully installed {package_with_version}")

# Only install non-standard library packages
install_package("boto3")       # For S3/MinIO operations (will install botocore)
install_package("h5py")        # For HDF5 file operations
install_package("matplotlib")  # For plotting
install_package("psutil")      # For memory tracking
install_package("numpy")       # For numerical operations

print("Dependencies installed successfully.")


Installing boto3...
Installing h5py...
Installing matplotlib...
Installing psutil...
Installing numpy...
Dependencies installed successfully.


In [ ]:
# 2_rom_modeling.ipynb
#
# This notebook preprocesses the Cylinder Flow Dataset with memory optimizations:
# - Loads the data from MinIO
# - Extracts velocity fields
# - Creates the snapshot matrix
# - Performs mean subtraction
# - Normalizes the data if needed
# - Uploads processed data back to MinIO

import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import json
from datetime import datetime
import boto3
from botocore.client import Config
import tempfile
import io
import gc  # For garbage collection
import psutil  # For memory tracking

# Memory management flags
MEMORY_EFFICIENT = True
CHUNK_SIZE = 10  # Process this many snapshots at a time
ENABLE_MEMORY_TRACKING = True
DOWNSAMPLE = False
DOWNSAMPLE_FACTOR = 2  # Only use every Nth point in grid

# Thread control to limit memory usage
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["NUMEXPR_MAX_THREADS"] = "2"

# Function to track memory usage
def print_memory_usage(label=""):
    if ENABLE_MEMORY_TRACKING:
        process = psutil.Process(os.getpid())
        memory_mb = process.memory_info().rss / 1024 / 1024
        print(f"Memory usage {label}: {memory_mb:.1f} MB")

print_memory_usage("at start")

# Function to convert NumPy types to Python native types for JSON serialization
def json_serialize(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, tuple) and hasattr(obj, '_asdict'):
        # Handle named tuples
        return obj._asdict()
    elif isinstance(obj, tuple):
        return list(obj)
    else:
        return obj

def save_dict_to_json(data_dict, filepath):
    """Save dictionary to JSON with NumPy value conversion"""
    # Convert data to JSON-serializable types
    serializable_dict = {}
    for key, value in data_dict.items():
        serializable_dict[key] = json_serialize(value)
    
    # Save to file
    with open(filepath, 'w') as f:
        json.dump(serializable_dict, f, indent=2)

# Create a temporary local directory for processing
temp_dir = tempfile.mkdtemp()
print(f"Using temporary directory: {temp_dir}")

# Connect to MinIO
print("Connecting to MinIO...")
s3_endpoint = os.environ.get('S3_ENDPOINT', 'http://minio.minio-system.svc.cluster.local:9000')

# Fix the endpoint URL if the protocol is missing
if s3_endpoint and not s3_endpoint.startswith(('http://', 'https://')):
    s3_endpoint = f"http://{s3_endpoint}"
    print(f"Adding http:// prefix to endpoint: {s3_endpoint}")

s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID', 'minio')
s3_secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY', 'minio123')

s3 = boto3.client('s3',
                  endpoint_url=s3_endpoint,
                  aws_access_key_id=s3_access_key,
                  aws_secret_access_key=s3_secret_key,
                  config=Config(signature_version='s3v4'))

# Load parameters from previous step
MINIO_BUCKET = 'rom-data'
MINIO_OUTPUT_PREFIX = 'rom-pipeline/outputs'

# Download parameters file from MinIO
try:
    params_key = f"{MINIO_OUTPUT_PREFIX}/params.json"
    params_path = os.path.join(temp_dir, 'params.json')
    s3.download_file(MINIO_BUCKET, params_key, params_path)
    
    with open(params_path, 'r') as f:
        params = json.load(f)
        
    print(f"Loaded parameters successfully")
except Exception as e:
    print(f"Error loading parameters, using defaults: {str(e)}")
    params = {
        "dataset_name": "cylinder",
        "minio_bucket": MINIO_BUCKET,
        "minio_output_prefix": MINIO_OUTPUT_PREFIX
    }

# Update parameters for this step
params.update({
    "preprocessing_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "preprocessing_options": {
        "mean_subtraction": True,
        "normalization": True,
        "memory_efficient": MEMORY_EFFICIENT,
        "chunk_size": CHUNK_SIZE,
        "downsample": DOWNSAMPLE,
        "downsample_factor": DOWNSAMPLE_FACTOR if DOWNSAMPLE else None
    }
})

# Verify the previous step completed
try:
    marker_key = f"{MINIO_OUTPUT_PREFIX}/data_download_completed.txt"
    s3.head_object(Bucket=MINIO_BUCKET, Key=marker_key)
    print("Previous step (data fetching) completed successfully")
except Exception as e:
    print(f"Warning: Previous step completion marker not found: {str(e)}")
    print("Continuing anyway...")

# List objects in the data directory to check what's available
print("\nListing files in MinIO data directory:")
data_prefix = f"{MINIO_OUTPUT_PREFIX}/data/"
response = s3.list_objects_v2(Bucket=MINIO_BUCKET, Prefix=data_prefix)

if 'Contents' not in response or len(response.get('Contents', [])) == 0:
    print(f"Warning: No data files found in {MINIO_BUCKET}/{data_prefix}")
    print("Please ensure that data files have been uploaded.")
    raise FileNotFoundError(f"No data files found in {MINIO_BUCKET}/{data_prefix}")

# Download data files from MinIO
print("\nDownloading data files from MinIO:")
for obj in response.get('Contents', []):
    if obj['Key'].endswith(('.h5', '.hdf5')):
        filename = os.path.basename(obj['Key'])
        local_path = os.path.join(temp_dir, filename)
        
        print(f"  Downloading {obj['Key']} to {local_path}")
        s3.download_file(MINIO_BUCKET, obj['Key'], local_path)

# Find HDF5 files
h5_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) 
            if f.endswith('.h5') or f.endswith('.hdf5')]

if not h5_files:
    raise FileNotFoundError(f"No HDF5 files found in downloaded data")

print(f"Found {len(h5_files)} HDF5 files: {[os.path.basename(f) for f in h5_files]}")

# Load velocity data from the first file
h5_file = h5_files[0]
print(f"Loading data from {os.path.basename(h5_file)}")

# Load the data or just get its shape
data = None  # We'll load data in chunks if memory efficient mode is on
shape = None
velocity_key = None

try:
    with h5py.File(h5_file, 'r') as f:
        # First, explore the HDF5 file structure to find the velocity field
        print("Exploring HDF5 file structure:")
        print("Top-level groups/datasets:")
        for key in f.keys():
            if isinstance(f[key], h5py.Group):
                print(f"  Group: {key} (contains: {list(f[key].keys())})")
            elif isinstance(f[key], h5py.Dataset):
                print(f"  Dataset: {key} (shape: {f[key].shape}, dtype: {f[key].dtype})")
        
        # Define possible velocity field names to check
        velocity_keys = ['velocity', 'u', 'v', 'vel', 'flow', 'flowfield', 'vector']
        
        # Search for velocity data
        velocity_key = None
        
        # Step 1: Check direct keys in root level
        for key in velocity_keys:
            if key in f:
                velocity_key = key
                print(f"Found direct velocity key: {key}")
                break
        
        # Step 2: Look for keys containing velocity terms
        if velocity_key is None:
            for key in f.keys():
                if any(vk in key.lower() for vk in velocity_keys):
                    velocity_key = key
                    print(f"Found key containing velocity term: {key}")
                    break
        
        # Step 3: Search in groups
        if velocity_key is None:
            for key in f.keys():
                if isinstance(f[key], h5py.Group):
                    for subkey in f[key].keys():
                        if any(vk == subkey.lower() or vk in subkey.lower() for vk in velocity_keys):
                            velocity_key = f"{key}/{subkey}"
                            print(f"Found velocity data in group: {velocity_key}")
                            break
        
        # Step 4: Look for large datasets (likely to be the flow field)
        if velocity_key is None:
            largest_dataset = None
            largest_size = 0
            
            def find_largest_dataset(name, obj):
                global largest_dataset, largest_size
                if isinstance(obj, h5py.Dataset):
                    # Skip small datasets (likely metadata)
                    if obj.size > 1000 and obj.size > largest_size:
                        largest_size = obj.size
                        largest_dataset = name
            
            # Traverse the entire file
            f.visititems(find_largest_dataset)
            
            if largest_dataset:
                velocity_key = largest_dataset
                print(f"Selected largest dataset as velocity data: {velocity_key} (size: {largest_size})")
        
        if velocity_key is None:
            raise KeyError("Could not find velocity data in the file")
            
        print(f"Loading velocity data from '{velocity_key}'")
        
        # Just get the shape if memory efficient
        if MEMORY_EFFICIENT:
            shape = f[velocity_key].shape
            print(f"Data shape: {shape} (memory efficient mode - not loading full data)")
        else:
            data = f[velocity_key][:]
            shape = data.shape
            print(f"Data shape: {shape}")
        
        # Load any coordinates or metadata if available
        x_coords = None
        y_coords = None
        time_data = None
        
        # Look for common coordinate names
        for coord_name in ['x', 'y', 'z', 'X', 'Y', 'Z', 'coord', 'coords', 'coordinates', 'grid', 'time', 't']:
            if coord_name in f:
                if coord_name.lower() in ['x', 'X']:
                    x_coords = f[coord_name][:]
                    print(f"Found x coordinates: shape {x_coords.shape}")
                elif coord_name.lower() in ['y', 'Y']:
                    y_coords = f[coord_name][:]
                    print(f"Found y coordinates: shape {y_coords.shape}")
                elif coord_name.lower() in ['time', 't']:
                    time_data = f[coord_name][:]
                    print(f"Found time data: shape {time_data.shape}")
                else:
                    print(f"Found other coordinate data '{coord_name}': shape {f[coord_name].shape}")
        
except Exception as e:
    print(f"Error loading data: {str(e)}")
    raise

print_memory_usage("after data exploration")

# Determine data dimensions
if len(shape) == 2:  # (time, points) or (points, time)
    if shape[0] < shape[1]:  # Likely (time, points)
        n_snapshots = shape[0]
        n_points = shape[1]
        n_dims = 1
        structured_grid = False
        print(f"Detected unstructured data: {n_snapshots} snapshots, {n_points} points, {n_dims} dimensions")
    else:  # Likely (points, time)
        n_snapshots = shape[1]
        n_points = shape[0]
        n_dims = 1
        structured_grid = False
        print(f"Detected unstructured data: {n_snapshots} snapshots, {n_points} points, {n_dims} dimensions")
        
elif len(shape) == 3:  # (time, x, y) or (time, points, dims)
    if shape[1] > shape[2]:  # Likely (time, points, dims)
        n_snapshots = shape[0]
        n_points = shape[1]
        n_dims = shape[2]
        structured_grid = False
        print(f"Detected (time, points, dimensions) format: {n_snapshots} snapshots, {n_points} points, {n_dims} dimensions")
    else:  # Likely (time, x, y) for a scalar field
        n_snapshots = shape[0]
        grid_shape = (shape[1], shape[2])
        n_points = grid_shape[0] * grid_shape[1]
        n_dims = 1
        structured_grid = True
        print(f"Detected structured scalar data: {n_snapshots} snapshots, grid shape {grid_shape}")
    
    else:
        raise ValueError(f"Could not determine data structure from shape {shape}")

elif len(shape) == 4:  # Likely (time, x, y, components) 
    if shape[0] < shape[1] and shape[0] < shape[2]:  # Typical for time series
        n_snapshots = shape[0]
        grid_shape = (shape[1], shape[2])
        n_points = np.prod(grid_shape)
        n_dims = shape[3]
        structured_grid = True
        print(f"Detected (time, x, y, components) format: {n_snapshots} snapshots, grid shape {grid_shape}, {n_dims} dimensions")
    else:
        raise ValueError(f"Could not determine data structure from shape {shape}")
else:
    raise ValueError(f"Unsupported data shape: {shape}")

# Create snapshot matrix and process mean/fluctuations
if MEMORY_EFFICIENT and data is None:
    print(f"Processing data in chunks of {CHUNK_SIZE} snapshots to save memory")
    # Initialize arrays for the final results
    snapshot_matrix = np.zeros((n_points * n_dims, n_snapshots))
    mean_flow = np.zeros((n_points * n_dims, 1))
    
    # Process data in chunks
    with h5py.File(h5_file, 'r') as f:
        # We'll compute the mean in a streaming fashion
        chunk_means = []
        chunk_weights = []

        for chunk_start in range(0, n_snapshots, CHUNK_SIZE):
            chunk_end = min(chunk_start + CHUNK_SIZE, n_snapshots)
            chunk_size = chunk_end - chunk_start
            print(f"Processing chunk {chunk_start//CHUNK_SIZE + 1} of {(n_snapshots-1)//CHUNK_SIZE + 1}: snapshots {chunk_start+1}-{chunk_end}")
            
            # Load this chunk of data
            if len(shape) == 4:  # (time, x, y, components)
                chunk_data = f[velocity_key][chunk_start:chunk_end]
                # Reshape to (points*dims, chunk_size)
                chunk_reshaped = np.zeros((n_points * n_dims, chunk_size))
                for i in range(chunk_size):
                    chunk_reshaped[:, i] = chunk_data[i].reshape(n_points, n_dims).flatten()
            elif len(shape) == 3:  # (time, x, y) for scalar field or (time, points, dims)
                chunk_data = f[velocity_key][chunk_start:chunk_end]
                # Reshape to (points*dims, chunk_size)
                chunk_reshaped = np.zeros((n_points * n_dims, chunk_size))
                
                if structured_grid:  # (time, x, y) for scalar field
                    for i in range(chunk_size):
                        # Reshape 2D grid to 1D array
                        chunk_reshaped[:, i] = chunk_data[i].reshape(n_points)
                else:  # (time, points, dims)
                    for i in range(chunk_size):
                        # Flatten the points and dimensions
                        chunk_reshaped[:, i] = chunk_data[i].reshape(n_points * n_dims)
            elif len(shape) == 2:  # (time, points) or (points, time)
                if shape[0] < shape[1]:  # (time, points)
                    chunk_data = f[velocity_key][chunk_start:chunk_end]
                    chunk_reshaped = chunk_data.T  # Transpose to (points, chunk_size)
                else:  # (points, time)
                    chunk_data = f[velocity_key][:, chunk_start:chunk_end]
                    chunk_reshaped = chunk_data  # Already (points, chunk_size)
            else:
                raise ValueError(f"Unsupported data shape for chunked processing: {shape}")
            
            # Copy to snapshot matrix
            snapshot_matrix[:, chunk_start:chunk_end] = chunk_reshaped
            
            # Compute chunk mean for streaming mean calculation
            chunk_mean = np.mean(chunk_reshaped, axis=1, keepdims=True)
            chunk_means.append(chunk_mean)
            chunk_weights.append(chunk_size / n_snapshots)
            
            # Free memory
            del chunk_data
            del chunk_reshaped
            gc.collect()
            print_memory_usage(f"after processing chunk {chunk_start//CHUNK_SIZE + 1}")
        
        # Compute overall mean from chunk means
        for i, (chunk_mean, weight) in enumerate(zip(chunk_means, chunk_weights)):
            mean_flow += chunk_mean * weight
        
        print(f"Computed mean flow with shape: {mean_flow.shape}")
        
        # Compute fluctuations
        print("Computing fluctuations...")
        fluctuations = np.zeros_like(snapshot_matrix)
        
        for chunk_start in range(0, n_snapshots, CHUNK_SIZE):
            chunk_end = min(chunk_start + CHUNK_SIZE, n_snapshots)
            print(f"Processing fluctuations for chunk {chunk_start//CHUNK_SIZE + 1}")
            
            # Subtract mean
            fluctuations[:, chunk_start:chunk_end] = snapshot_matrix[:, chunk_start:chunk_end] - mean_flow
            
            print_memory_usage(f"after fluctuations for chunk {chunk_start//CHUNK_SIZE + 1}")
else:
    # Process all data at once
    print("Processing all data at once")
    
    # Create snapshot matrix - reshape data into a 2D matrix
    # For POD, we need a matrix of shape (n_points*n_dims, n_snapshots)
    if structured_grid:
        if n_dims == 1:
            # For scalar field
            snapshot_matrix = np.zeros((n_points, n_snapshots))
            for i in range(n_snapshots):
                snapshot_matrix[:, i] = data[i].reshape(n_points)
        else:
            # For vector field
            snapshot_matrix = np.zeros((n_points * n_dims, n_snapshots))
            for i in range(n_snapshots):
                # Reshape and stack components
                reshaped = data[i].reshape(n_points, n_dims)
                snapshot_matrix[:, i] = reshaped.flatten()
    else:
        # Already in the right format for unstructured grid
        if n_dims == 1:
            if shape[0] < shape[1]:
                snapshot_matrix = data.T  # Transpose to get (n_points, n_snapshots)
            else:
                snapshot_matrix = data  # Already (n_points, n_snapshots)
        else:
            snapshot_matrix = np.zeros((n_points * n_dims, n_snapshots))
            for i in range(n_snapshots):
                snapshot_matrix[:, i] = data[i].flatten()

    print(f"Created snapshot matrix with shape: {snapshot_matrix.shape}")
    print_memory_usage("after creating snapshot matrix")

    # Compute mean flow
    mean_flow = np.mean(snapshot_matrix, axis=1, keepdims=True)
    print(f"Mean flow shape: {mean_flow.shape}")

    # Subtract mean (centering the data)
    fluctuations = snapshot_matrix - mean_flow
    print("Mean subtraction applied")
    print_memory_usage("after mean subtraction")

# Normalize the data if needed
if params["preprocessing_options"].get("normalization", True):
    # Compute the Frobenius norm of the fluctuation matrix
    if MEMORY_EFFICIENT:
        # Compute norm in chunks
        frob_norm_sq = 0
        for chunk_start in range(0, n_snapshots, CHUNK_SIZE):
            chunk_end = min(chunk_start + CHUNK_SIZE, n_snapshots)
            chunk = fluctuations[:, chunk_start:chunk_end]
            frob_norm_sq += np.sum(chunk**2)
        frob_norm = np.sqrt(frob_norm_sq)
    else:
        frob_norm = np.linalg.norm(fluctuations)
    
    # Normalize
    fluctuations = fluctuations / frob_norm
    print(f"Normalization applied (Frobenius norm: {frob_norm:.4f})")
    
    # Save the normalization factor for later use
    params["preprocessing_options"]["normalization_factor"] = float(frob_norm)
else:
    print("Normalization skipped")

print_memory_usage("after normalization")

# Compute SVD for POD modes
print("\nComputing SVD for POD modes...")
try:
    # Use randomized SVD for large datasets
    if n_points * n_dims > 10000 or n_snapshots > 1000:
        from sklearn.decomposition import TruncatedSVD
        n_modes = min(n_snapshots - 1, 100)  # Limit to 100 modes for very large datasets
        print(f"Using randomized SVD with {n_modes} modes due to large data size")
        
        svd = TruncatedSVD(n_components=n_modes, random_state=42)
        svd.fit(fluctuations.T)  # TruncatedSVD expects (n_samples, n_features)
        
        # Extract components
        spatial_modes = svd.components_.T  # Shape: (n_points*n_dims, n_modes)
        singular_values = svd.singular_values_
        temporal_modes = fluctuations.T @ spatial_modes / singular_values  # Shape: (n_snapshots, n_modes)
    else:
        # Use full SVD for smaller datasets
        U, S, Vt = np.linalg.svd(fluctuations, full_matrices=False)
        
        # Extract components
        spatial_modes = U  # Shape: (n_points*n_dims, n_modes)
        singular_values = S
        temporal_modes = Vt.T  # Shape: (n_snapshots, n_modes)
        
    print(f"SVD completed successfully")
    print(f"Spatial modes shape: {spatial_modes.shape}")
    print(f"Singular values shape: {singular_values.shape}")
    print(f"Temporal modes shape: {temporal_modes.shape}")
    
    # Compute energy content
    energy = singular_values**2
    total_energy = np.sum(energy)
    energy_ratio = energy / total_energy
    cumulative_energy = np.cumsum(energy_ratio)
    
    print(f"Total energy: {total_energy:.4f}")
    print(f"Energy in first mode: {energy[0]:.4f} ({energy_ratio[0]*100:.2f}%)")
    
    # Find number of modes for different energy thresholds
    for threshold in [0.9, 0.95, 0.99, 0.999]:
        n_modes = np.argmax(cumulative_energy >= threshold) + 1
        print(f"Number of modes for {threshold*100:.1f}% energy: {n_modes}")
    
    # Save results
    pod_results = {
        'singular_values': singular_values,
        'energy_ratio': energy_ratio,
        'cumulative_energy': cumulative_energy,
        'n_points': n_points,
        'n_dims': n_dims,
        'n_snapshots': n_snapshots,
        'structured_grid': structured_grid
    }
    
    if structured_grid:
        pod_results['grid_shape'] = grid_shape
    
    # Save modes data
    modes_data = {
        'spatial_modes': spatial_modes,
        'temporal_modes': temporal_modes,
        'singular_values': singular_values,
        'mean_flow': mean_flow
    }
    
    # Save to NPZ file
    pod_results_file = os.path.join(temp_dir, 'pod_results.npz')
    print(f"Saving POD results to {pod_results_file}")
    np.savez_compressed(pod_results_file, **pod_results)
    
    modes_data_file = os.path.join(temp_dir, 'pod_modes.npz')
    print(f"Saving POD modes to {modes_data_file}")
    np.savez_compressed(modes_data_file, **modes_data)
    
    # Upload results to MinIO
    pod_results_key = f"{MINIO_OUTPUT_PREFIX}/pod_results.npz"
    print(f"Uploading POD results to {MINIO_BUCKET}/{pod_results_key}")
    s3.upload_file(pod_results_file, MINIO_BUCKET, pod_results_key)
    
    modes_data_key = f"{MINIO_OUTPUT_PREFIX}/pod_modes.npz"
    print(f"Uploading POD modes to {MINIO_BUCKET}/{modes_data_key}")
    s3.upload_file(modes_data_file, MINIO_BUCKET, modes_data_key)
    
    # Visualize energy content
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.semilogy(range(1, len(singular_values)+1), singular_values, 'o-')
    plt.grid(True)
    plt.xlabel('Mode number')
    plt.ylabel('Singular value')
    plt.title('Singular Values Decay')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(cumulative_energy)+1), cumulative_energy, 'o-')
    plt.grid(True)
    plt.xlabel('Number of modes')
    plt.ylabel('Cumulative energy ratio')
    plt.title('Cumulative Energy Content')
    
    # Add horizontal lines for thresholds
    for threshold in [0.9, 0.95, 0.99]:
        plt.axhline(y=threshold, color='r', linestyle='--', alpha=0.5)
        plt.text(len(cumulative_energy)*0.8, threshold+0.01, f'{threshold*100:.0f}%', color='r')
    
    plt.tight_layout()
    
    # Save visualization
    energy_plot = os.path.join(temp_dir, 'energy_plot.png')
    plt.savefig(energy_plot, dpi=150)
    
    # Upload visualization
    energy_plot_key = f"{MINIO_OUTPUT_PREFIX}/visualizations/energy_plot.png"
    print(f"Uploading energy plot to {MINIO_BUCKET}/{energy_plot_key}")
    s3.upload_file(energy_plot, MINIO_BUCKET, energy_plot_key)
    
    # Visualize a few spatial modes
    if structured_grid:
        n_vis_modes = min(4, spatial_modes.shape[1])
        plt.figure(figsize=(15, 3*n_vis_modes))
        
        for i in range(n_vis_modes):
            plt.subplot(n_vis_modes, 1, i+1)
            
            if n_dims == 1:
                # Scalar field
                mode_reshaped = spatial_modes[:, i].reshape(grid_shape)
                plt.imshow(mode_reshaped, cmap='RdBu_r')
                plt.colorbar(label=f'Mode {i+1}')
            else:
                # Vector field - plot magnitude
                mode_reshaped = spatial_modes[:, i].reshape(grid_shape[0], grid_shape[1], n_dims)
                mode_magnitude = np.sqrt(np.sum(mode_reshaped**2, axis=2))
                plt.imshow(mode_magnitude, cmap='viridis')
                plt.colorbar(label=f'Mode {i+1} Magnitude')
                
            plt.title(f'Spatial Mode {i+1} (Energy: {energy_ratio[i]*100:.2f}%)')
            plt.xlabel('X')
            plt.ylabel('Y')
        
        plt.tight_layout()
        
        # Save visualization
        modes_plot = os.path.join(temp_dir, 'spatial_modes.png')
        plt.savefig(modes_plot, dpi=150)
        
        # Upload visualization
        modes_plot_key = f"{MINIO_OUTPUT_PREFIX}/visualizations/spatial_modes.png"
        print(f"Uploading spatial modes plot to {MINIO_BUCKET}/{modes_plot_key}")
        s3.upload_file(modes_plot, MINIO_BUCKET, modes_plot_key)
    
    # Visualize temporal modes
    n_vis_modes = min(4, temporal_modes.shape[1])
    plt.figure(figsize=(15, 2*n_vis_modes))
    
    for i in range(n_vis_modes):
        plt.subplot(n_vis_modes, 1, i+1)
        plt.plot(temporal_modes[:, i])
        plt.grid(True)
        plt.title(f'Temporal Mode {i+1} (Energy: {energy_ratio[i]*100:.2f}%)')
        plt.xlabel('Time step')
        plt.ylabel('Amplitude')
    
    plt.tight_layout()
    
    # Save visualization
    temporal_plot = os.path.join(temp_dir, 'temporal_modes.png')
    plt.savefig(temporal_plot, dpi=150)
    
    # Upload visualization
    temporal_plot_key = f"{MINIO_OUTPUT_PREFIX}/visualizations/temporal_modes.png"
    print(f"Uploading temporal modes plot to {MINIO_BUCKET}/{temporal_plot_key}")
    s3.upload_file(temporal_plot, MINIO_BUCKET, temporal_plot_key)
    
    # Update parameters with POD results
    params.update({
        "pod_results": {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "n_modes": len(singular_values),
            "energy_thresholds": {
                "90%": int(np.argmax(cumulative_energy >= 0.9) + 1),
                "95%": int(np.argmax(cumulative_energy >= 0.95) + 1),
                "99%": int(np.argmax(cumulative_energy >= 0.99) + 1),
                "99.9%": int(np.argmax(cumulative_energy >= 0.999) + 1)
            },
            "first_mode_energy": float(energy_ratio[0])
        }
    })
    
except Exception as e:
    print(f"Error computing SVD: {str(e)}")
    # Continue with the rest of the script
    params.update({
        "pod_results": {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "error": str(e)
        }
    })

print_memory_usage("after SVD computation")

# Save parameters
params_file = os.path.join(temp_dir, 'params.json')
save_dict_to_json(params, params_file)

# Upload parameters to MinIO
params_key = f"{MINIO_OUTPUT_PREFIX}/params.json"
print(f"Uploading parameters to {MINIO_BUCKET}/{params_key}")
s3.upload_file(params_file, MINIO_BUCKET, params_key)

# Create a completion marker
completion_marker = os.path.join(temp_dir, 'rom_modeling_completed.txt')
with open(completion_marker, 'w') as f:
    f.write(f"ROM modeling completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Processed {n_snapshots} snapshots with {n_points} spatial points and {n_dims} dimensions\n")
    if 'pod_results' in params and 'error' not in params['pod_results']:
        f.write(f"Computed {len(singular_values)} POD modes\n")
        f.write(f"90% energy captured with {params['pod_results']['energy_thresholds']['90%']} modes\n")
        f.write(f"99% energy captured with {params['pod_results']['energy_thresholds']['99%']} modes\n")

# Upload completion marker to MinIO
marker_key = f"{MINIO_OUTPUT_PREFIX}/rom_modeling_completed.txt"
print(f"Uploading completion marker to {MINIO_BUCKET}/{marker_key}")
s3.upload_file(completion_marker, MINIO_BUCKET, marker_key)

print("\nROM modeling completed successfully!")
print(f"Processed {n_snapshots} snapshots with {n_points} spatial points and {n_dims} dimensions")
if 'pod_results' in params and 'error' not in params['pod_results']:
    print(f"Computed {len(singular_values)} POD modes")
    print(f"90% energy captured with {params['pod_results']['energy_thresholds']['90%']} modes")
    print(f"99% energy captured with {params['pod_results']['energy_thresholds']['99%']} modes")
print(f"Results saved to {MINIO_BUCKET}/{MINIO_OUTPUT_PREFIX}/")
